In [1]:
import pandas as pd
import sqlite3
import json

#get data from kismet db
#devices table has information about each device, the device entry of the devices table has json of each device
#packets will have "every occurrence" of a device
#match up the two on the devices.devmac and packets.sourcemac
sql = '''SELECT packets.sourcemac,
                packets.lat,
                packets.lon,
                packets.ts_sec,
                devices.type,
                devices.device
         FROM packets
         INNER JOIN devices
         ON packets.sourcemac=devices.devmac;'''
         
conn = sqlite3.connect('e:/kismet_db/2018_05_19_fairfield_herndon_to_tyson_corner_center.kismet')
kismet = pd.read_sql_query(sql, conn)

In [2]:
#drop the duplicates and convert the ts_sec to an actual datetime
kismet.drop_duplicates(inplace=True)
kismet.ts_sec = pd.to_datetime(kismet.ts_sec, unit='s')

In [3]:
#just example function to pull something from the json and assign to a column
#in this case, it's the device's manufacturer/vendor
def get_vendor(dev):
    dev_json = json.loads(dev)
    return dev_json.get('kismet.device.base.manuf', 'Unknown')

In [4]:
#applying the get_vendor function to a new column
kismet['vendor'] = kismet.device.apply(get_vendor)

In [5]:
#remove anything that doesn't have coordinates
#btw, if you are in static-monitoring positions
#recommend you use the virtual gps (i.e, fixed coordinates) option in kismet
kismet = kismet[(kismet.lat != 0) & (kismet.lon != 0)]

In [6]:
#the old kismet dbs (version 4) had the coordinates as integers, not required in new kismet db (version 5)
#basically, if the coords are floats, you don't need this
def make_float(coord):
    coord = coord / 100000
    return coord

In [7]:
#apply the make_float function to the lats and longs
kismet.lat = kismet.lat.apply(make_float)
kismet.lon = kismet.lon.apply(make_float)

In [8]:
kismet.head(10)

,sourcemac,lat,lon,ts_sec,type,device,vendor
127,D8:38:FC:46:99:CC,38.96822,-77.37845,2018-05-18 18:55:46,Wi-Fi AP,"{""kismet.device.base.key"": ""4202770DE306B141_C...",RuckusWi
128,54:78:1A:0C:2D:4F,38.96822,-77.37845,2018-05-18 18:55:46,Wi-Fi AP,"{""kismet.device.base.key"": ""4202770DE306B141_4...",Cisco
129,54:78:1A:0C:2D:4E,38.96822,-77.37845,2018-05-18 18:55:47,Wi-Fi AP,"{""kismet.device.base.key"": ""4202770DE306B141_4...",Cisco
130,AC:3A:7A:7E:77:0F,38.96822,-77.37845,2018-05-18 18:55:47,Wi-Fi AP,"{""kismet.device.base.key"": ""4202770DE306B141_F...",Roku
131,D8:38:FC:06:8B:58,38.96822,-77.37845,2018-05-18 18:55:47,Wi-Fi AP,"{""kismet.device.base.key"": ""4202770DE306B141_5...",RuckusWi
132,D8:38:FC:46:8B:58,38.96822,-77.37845,2018-05-18 18:55:47,Wi-Fi AP,"{""kismet.device.base.key"": ""4202770DE306B141_5...",RuckusWi
133,02:FB:6A:B0:7B:00,38.96822,-77.37845,2018-05-18 18:55:47,Wi-Fi Client,"{""kismet.device.base.key"": ""4202770DE306B141_7...",Unknown
134,D8:38:FC:06:96:68,38.96822,-77.37845,2018-05-18 18:55:47,Wi-Fi AP,"{""kismet.device.base.key"": ""4202770DE306B141_6...",RuckusWi
135,D8:38:FC:46:96:68,38.96822,-77.37845,2018-05-18 18:55:47,Wi-Fi AP,"{""kismet.device.base.key"": ""4202770DE306B141_6...",RuckusWi
137,D8:38:FC:06:B3:08,38.96822,-77.37845,2018-05-18 18:55:47,Wi-Fi AP,"{""kismet.device.base.key"": ""4202770DE306B141_8...",RuckusWi
